##### Imports

In [412]:
import pandas as pd

In [413]:
data = pd.read_csv('../data/train.csv', index_col='Id')
print(data.shape)
data.head()

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


##### Handling missing values

In [414]:
print("; ".join(data.isnull().mean().round(4).mul(100).sort_values(ascending=False).to_string().split("\n")))

PoolQC           99.52; MiscFeature      96.30; Alley            93.77; Fence            80.75; FireplaceQu      47.26; LotFrontage      17.74; GarageYrBlt       5.55; GarageCond        5.55; GarageType        5.55; GarageFinish      5.55; GarageQual        5.55; BsmtExposure      2.60; BsmtFinType2      2.60; BsmtCond          2.53; BsmtQual          2.53; BsmtFinType1      2.53; MasVnrArea        0.55; MasVnrType        0.55; Electrical        0.07; MSSubClass        0.00; Fireplaces        0.00; Functional        0.00; KitchenQual       0.00; KitchenAbvGr      0.00; BedroomAbvGr      0.00; HalfBath          0.00; FullBath          0.00; BsmtHalfBath      0.00; TotRmsAbvGrd      0.00; GarageCars        0.00; GrLivArea         0.00; GarageArea        0.00; PavedDrive        0.00; WoodDeckSF        0.00; OpenPorchSF       0.00; EnclosedPorch     0.00; 3SsnPorch         0.00; ScreenPorch       0.00; PoolArea          0.00; MiscVal           0.00; MoSold            0.00; YrSold          

PoolQC, MiscFeature, Alley, Fence, FireplaceQu, LotFrontage have a high percentage of missing values so we're dropping them.
GarageCond, GarageType, GarageFinish, GarageQual have connected missing values - they're missing when there's no garage on the property *and* are extremely correlated with GarageCars, GarageArea, so we can safely drop them instead of filling them.

In [415]:
to_drop = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage', 'GarageFinish', 'GarageQual', 'GarageCond', 'GarageType']
data.drop(labels=to_drop, axis=1, inplace=True)

When the property is missing a basement there's no entry in the data for the basement columns. We're just going to fill them with 'No Basement'. I assume it's the same for the masonry type and area since their missing values are correlated and we're going to fill them with 'No Masonry' and 0 accordingly. The year when the Garage is built is probably missing when there's no garage so we're filling it with the year the house was built instead. The Electrical col only has 0.07% missing values and since we don't have information on why the data is missing we're going to drop the rows.

In [416]:
data['BsmtFinType2'].fillna('No Basement', inplace=True)
data['BsmtExposure'].fillna('No Basement', inplace=True)
data['BsmtCond'].fillna('No Basement', inplace=True)
data['BsmtFinType1'].fillna('No Basement', inplace=True)
data['BsmtQual'].fillna('No Basement', inplace=True)
data['MasVnrType'].fillna('No Masonry', inplace=True)
data['MasVnrArea'].fillna(0, inplace=True)
data.loc[data['GarageYrBlt'].isna(), 'GarageYrBlt'] = data['YearBuilt'][data['GarageYrBlt'].isna()]
data.dropna(subset=['Electrical'], axis=0, inplace=True)

##### Separating the numerical and categorical columns

*One small detail, MSSubClass is registered as a number in the DataFrame, it is a categorical variables so I'll convert it to object.*

In [417]:
data['MSSubClass'] = data['MSSubClass'].astype('object')

*Now we can split them correctly...*

In [418]:
import warnings
warnings.filterwarnings('ignore')

numerical = [col for col in data.columns if data[col].dtype in ('Int64', 'Float64')]
categorical = [col for col in data.columns if data[col].dtype not in ('Int64', 'Float64')]
print('Numerical: ', numerical, '\nCategorical: ', categorical)

Numerical:  ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'] 
Categorical:  ['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']


##### One hot encoding

In [419]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(encoder.fit_transform(data[categorical]))
OH_cols.index = data.index
data = pd.concat([data.drop(categorical, axis=1), OH_cols], axis=1)
data.head()

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,226,227,228,229,230,231,232,233,234,235
Id,,,,,,,,,,,,,,,,,,,,,
1,8450,7,5,2003,2003,196.0,706,0,150,856,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,9600,6,8,1976,1976,0.0,978,0,284,1262,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,11250,7,5,2001,2002,162.0,486,0,434,920,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,9550,7,5,1915,1970,0.0,216,0,540,756,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,14260,8,5,2000,2000,350.0,655,0,490,1145,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


##### Scaling the numerical Data

###### optional: remove the outliers, this removes over 300 rows and I'm not sure if it's a good idea.

In [420]:
from scipy import stats
import numpy as np

numerical_X = [n for n in numerical if n != 'SalePrice']

print('Before: ', data.shape)
no_outlier_data = data[(np.abs(stats.zscore(data[numerical_X])) <= 3.5).all(axis=1)] 
print('After: ', no_outlier_data.shape)

Before:  (1459, 271)
After:  (1116, 271)


###### Split before scaling

In [421]:
from sklearn.model_selection import train_test_split

X, y = data.drop('SalePrice', axis=1), data['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1167, 270) (1167,) (292, 270) (292,)


In [422]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

SS_train_cols = pd.DataFrame(scaler.fit_transform(X_train[numerical_X]))
SS_test_cols = pd.DataFrame(scaler.transform(X_test[numerical_X]))

SS_train_cols.index = X_train.index
SS_test_cols.index = X_test.index

SS_train_cols.columns = X_train[numerical_X].columns
SS_test_cols.columns = X_test[numerical_X].columns

X_train = pd.concat([SS_train_cols, X_train.drop(numerical_X, axis=1)], axis=1)
X_test = pd.concat([SS_test_cols, X_test.drop(numerical_X, axis=1)], axis=1)

X_train.head()


,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,226,227,228,229,230,231,232,233,234,235
Id,,,,,,,,,,,,,,,,,,,,,
93,0.243094,-0.798634,1.326191,-1.699834,1.031893,-0.563927,0.560508,-0.28548,-0.923743,-0.424952,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
839,-0.111234,-0.798634,0.411496,0.782650,1.031893,-0.563927,-0.978774,-0.28548,1.229738,0.083332,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
361,-0.294634,-0.067649,0.411496,0.212350,-0.333490,-0.563927,0.690041,-0.28548,-1.034178,-0.397600,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
666,0.042970,1.394321,-0.503200,0.950386,0.739311,-0.345557,-0.978774,-0.28548,1.936062,0.783078,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
41,-0.191339,-0.067649,-0.503200,-0.223762,-0.967418,-0.012542,0.409386,-0.28548,-0.274938,0.058260,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


###### Scaling the dataset without the split for the final training

In [423]:
scaler2 = StandardScaler()

SS_cols = pd.DataFrame(scaler2.fit_transform(X[numerical_X]))
SS_cols.index = X.index
SS_cols.columns = X[numerical_X].columns

X = pd.concat([SS_cols, X.drop(numerical_X, axis=1)], axis=1)

X.head()


,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,226,227,228,229,230,231,232,233,234,235
Id,,,,,,,,,,,,,,,,,,,,,
1,-0.207125,0.650852,-0.517424,1.051899,0.879449,0.513594,0.574748,-0.28876,-0.944607,-0.460570,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.091909,-0.072372,2.178727,0.157540,-0.428864,-0.571009,1.171304,-0.28876,-0.641330,0.465629,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.073401,0.650852,-0.517424,0.985650,0.830993,0.325449,0.092239,-0.28876,-0.301842,-0.314568,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.096918,0.650852,-0.517424,-1.863050,-0.719600,-0.571009,-0.499931,-0.28876,-0.061936,-0.688698,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.374967,1.374077,-0.517424,0.952526,0.734081,1.365783,0.462893,-0.28876,-0.175099,0.198720,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


##### Processing the test data

In [424]:
test = pd.read_csv('../data/test.csv', index_col='Id')
test.head()

# handle missing values

test.drop(labels=to_drop, axis=1, inplace=True)
test['BsmtFinType2'].fillna('No Basement', inplace=True)
test['BsmtExposure'].fillna('No Basement', inplace=True)
test['BsmtCond'].fillna('No Basement', inplace=True)
test['BsmtFinType1'].fillna('No Basement', inplace=True)
test['BsmtQual'].fillna('No Basement', inplace=True)
test['MasVnrType'].fillna('No Masonry', inplace=True)
test['MasVnrArea'].fillna(0, inplace=True)
test.loc[test['GarageYrBlt'].isna(), 'GarageYrBlt'] = test['YearBuilt'][test['GarageYrBlt'].isna()]

# don't do this, 
# data.dropna(subset=['Electrical'], axis=0, inplace=True)
# we need every row - do this instead: 

test['Electrical'].fillna('SBrkr')

# BsmtFullBath     
# 0.14; BsmtHalfBath     
# 0.14; GarageArea       0.07; 
# BsmtFinSF1       0.07; 
# BsmtFinSF2       0.07; 
# BsmtUnfSF        0.07; 
# TotalBsmtSF      0.07; GarageCars

extra_missing = ['BsmtFullBath', 'BsmtHalfBath', 'GarageArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'GarageCars']
for m in extra_missing:
    test[m].fillna(0, inplace=True)

# SBrkr is the mode 

# convert MSSubClass

test['MSSubClass'] = test['MSSubClass'].astype('object')

# OHE

OH_testdata_cols = pd.DataFrame(encoder.transform(test[categorical]))
OH_testdata_cols.index = test.index
test = pd.concat([test.drop(categorical, axis=1), OH_testdata_cols], axis=1)

# Std Scaler

SS_testdata_cols = pd.DataFrame(scaler.transform(test[numerical_X]))
SS_testdata_cols.index = test.index
SS_testdata_cols.columns = test[numerical_X].columns
test = pd.concat([SS_testdata_cols, test.drop(numerical_X, axis=1)], axis=1)

print(test.shape)
test.head()


(1459, 270)


,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,226,227,228,229,230,231,232,233,234,235
Id,,,,,,,,,,,,,,,,,,,,,
1461,0.082515,-0.798634,0.411496,-0.357951,-1.162473,-0.563927,0.031582,0.636908,-0.677565,-0.411276,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1462,0.326894,-0.067649,0.411496,-0.458592,-1.308764,0.025673,1.013873,-0.285480,-0.364666,0.607571,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1463,0.286519,-0.798634,-0.503200,0.849745,0.641784,-0.563927,0.728901,-0.285480,-0.983562,-0.306428,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1464,-0.069380,-0.067649,0.411496,0.883292,0.641784,-0.454742,0.320872,-0.285480,-0.553326,-0.310987,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1465,-0.528851,1.394321,-0.503200,0.682009,0.349201,-0.563927,-0.410988,-0.285480,1.041078,0.495886,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


##### Exporting

In [425]:
# Ovie gi koristime pri evaluacija na modelite
X_train.to_csv('../out/t_x_train.csv', index=None)
X_test.to_csv('../out/t_x_test.csv', index=None)
y_train.to_csv('../out/t_y_train.csv', index=None)
y_test.to_csv('../out/t_y_test.csv', index=None)

# Ovie gi koristime koga kje go odberime najdobriot model za na kaggle natprevarot
X.to_csv('../out/train_X.csv', index=None)
y.to_csv('../out/train_Y.csv', index=None)

# Ova e kaggle test data
test.to_csv('../out/kaggle_test_X.csv', index='Id')